In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/indian-startup-funding/startup_funding.csv


In [41]:
df = pd.read_csv('/kaggle/input/indian-startup-funding/startup_funding.csv')
df.head()

,Sr No,Date dd/mm/yyyy,Startup Name,Industry Vertical,SubVertical,City Location,Investors Name,InvestmentnType,Amount in USD,Remarks
0,1,09/01/2020,BYJU’S,E-Tech,E-learning,Bengaluru,Tiger Global Management,Private Equity Round,"20,00,00,000",NaN
1,2,13/01/2020,Shuttl,Transportation,App based shuttle service,Gurgaon,Susquehanna Growth Equity,Series C,"80,48,394",NaN
2,3,09/01/2020,Mamaearth,E-commerce,Retailer of baby and toddler products,Bengaluru,Sequoia Capital India,Series B,"1,83,58,860",NaN
3,4,02/01/2020,https://www.wealthbucket.in/,FinTech,Online Investment,New Delhi,Vinod Khatumal,Pre-series A,"30,00,000",NaN
4,5,02/01/2020,Fashor,Fashion and Apparel,Embroiled Clothes For Women,Mumbai,Sprout Venture Partners,Seed Round,"18,00,000",NaN


Lets see the columns (features) we have available.


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3044 entries, 0 to 3043
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Sr No              3044 non-null   int64 
 1   Date dd/mm/yyyy    3044 non-null   object
 2   Startup Name       3044 non-null   object
 3   Industry Vertical  2873 non-null   object
 4   SubVertical        2108 non-null   object
 5   City  Location     2864 non-null   object
 6   Investors Name     3020 non-null   object
 7   InvestmentnType    3040 non-null   object
 8   Amount in USD      2084 non-null   object
 9   Remarks            419 non-null    object
dtypes: int64(1), object(9)
memory usage: 237.9+ KB


From the above output, we can see that there are columns with missing values, so we need to either fill them up using a method or just ignore them. Let's see the process of cleaning the data

Features Descriptions

-Sno: Serial number

-Date: Date of funding

-StartupName: Name of the startup

-IndustryVertical: Industry to which the startup belongs.

-SubVertical: Sub-category of the industry type.

-CityLocation: City which the startup is based out of.

-InvestorsName: Name of the investors involved in the funding round.

-InvestmentType: Either Private Equity or Seed Funding.

-AmountInUSD: Funding Amount in USD.

-Remarks: Other information, if any

**Data Cleaning**

In [43]:
#Checking percentage of missing values in each column
(df.isna().sum()/df.shape[0]*100 ).sort_values(ascending=False)

Remarks              86.235217
Amount in USD        31.537451
SubVertical          30.749014
City  Location        5.913272
Industry Vertical     5.617608
Investors Name        0.788436
InvestmentnType       0.131406
Sr No                 0.000000
Date dd/mm/yyyy       0.000000
Startup Name          0.000000
dtype: float64

In [44]:
#rename columns name
df.rename(columns={
  'Date dd/mm/yyyy':'date',
  'Startup Name':'startup',
  'Industry Vertical':'vertical',
  'SubVertical':'subvertical',
  'City  Location':'city',
  'Investors Name':'investors',
  'InvestmentnType':'round',
  'Amount in USD':'amount'
    
},inplace=True)
df.columns

Index(['Sr No', 'date', 'startup', 'vertical', 'subvertical', 'city',
       'investors', 'round', 'amount', 'Remarks'],
      dtype='object')

Now, lets go one by one and try to fill out missing values in each column.

*Remark*

In [45]:
#since remarks column have more than 80% of missing information we will drop this column
df.drop(columns=['Remarks'],inplace=True)

In [46]:
df.set_index('Sr No',inplace=True)

In [47]:
df['amount'].head()

Sr No
1    20,00,00,000
2       80,48,394
3     1,83,58,860
4       30,00,000
5       18,00,000
Name: amount, dtype: object

We see that the column is not in numeric format, so lets correct the format and then try to fill up the missing values



In [48]:
#we wil lreplace all strings and nan values in amount columns with zero
df['amount'] = df['amount'].fillna('0')

In [49]:
#operations to convert into a numerical value
df['amount'] = df['amount'].str.replace(',','')
df['amount'] = df['amount'].str.replace('undisclosed','0')
df['amount'] = df['amount'].str.replace('unknown','0')
df['amount'] = df['amount'].str.replace('Undisclosed','0')

In [50]:
#Masking the amount column to extract on numerical values
df = df[df['amount'].str.isdigit()]
#change datatype from string to float 
df['amount'] = df['amount'].astype('float')

In [51]:
#convert usd to rupees
def to_inr(x):
    return round(x*82.6/10000000,2)
df['amount'] = df['amount'].apply(to_inr)
df['amount']

Sr No
1       1652.00
2         66.48
3        151.64
4         24.78
5         14.87
         ...   
3040      37.17
3041       6.81
3042      12.39
3043       0.00
3044       1.16
Name: amount, Length: 3029, dtype: float64

*Date*
When i initially tried to convert Date column to date time, several errors popped up due to bad format ('.' instead of '/' and so on. So cleaned the strings, then converted to datetime

In [52]:
#Remove errors from date columns
df['date'] = df['date'].str.replace('05/072018','05/07/2018')
df['date'] = pd.to_datetime(df['date'],errors='coerce')


In [54]:
df = df.dropna(subset=['date','startup','vertical','city','investors','round','amount'])

In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2822 entries, 1 to 2873
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         2822 non-null   datetime64[ns]
 1   startup      2822 non-null   object        
 2   vertical     2822 non-null   object        
 3   subvertical  2078 non-null   object        
 4   city         2822 non-null   object        
 5   investors    2822 non-null   object        
 6   round        2822 non-null   object        
 7   amount       2822 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 198.4+ KB


In [56]:
df.to_csv('startup_cleaned.csv',index=False)

*Subvertical*

In [67]:
len(df['subvertical'].unique())
df['subvertical'].shape[0]-len(df['subvertical'].unique())

904

In [64]:
df['subvertical'].describe()

count                        2078
unique                       1917
top       Online Lending Platform
freq                           11
Name: subvertical, dtype: object

There are 1918 unique sub-categories and 904 unknown sub-categories

We have two options from here, that is,

Fill up the missing values as 'Not specified' string
Do not fill up the missing values.
For this analysis, I intend to not fill up the missing values for the SubVertical columns since later on, it might affect our analysis.